#### This is the notebook I used to build up the libary's core funcion.
#### I also used it to read the gff3 file from genecode project to create the feature dataframe and save as parquet file in the resources folder.


In [34]:

from io import StringIO
from os import PathLike
from pathlib import Path
import gzip
import logging
import pandas as pd


def read_gff3_to_df(feature_file_path: PathLike) -> pd.DataFrame:
    # Define the columns in the GFF3 file
    GFF3_columns=["Sequence ID","source","Feature Type","Feature Start","Feature End","Score","Strand","Phase","INFO"]
    
    # Check if the file is gzipped or not
    if feature_file_path.suffix==".gz":
        FILE=gzip.open(feature_file_path, 'r')
    elif feature_file_path.suffix==".gff3" or feature_file_path.suffix==".gff":
        FILE=open(feature_file_path, 'r')
    else:
        raise ValueError(f"The file {feature_file_path} is not a recognized file type. Please provide a .gz, .gff3 or .gff file.")
        
    with FILE as file:
        outlist=list()
        for line in file.readlines():
            line=line.decode().strip()
            if not line.startswith("#"):
                # Parse the line and create a dictionary
                parsed_dict=dict(zip(GFF3_columns,line.split("\t")))
                # Parse the INFO field and create a dictionary
                INFO_dict=dict([item.split("=") for item in parsed_dict["INFO"].split(";")])
                # Update the parsed dictionary with the INFO dictionary
                parsed_dict.update(INFO_dict)
                parsed_dict.pop("INFO")
                outlist.append(parsed_dict)

    return pd.DataFrame(outlist)

# Define the file paths for GRCh38 and GRCh37 GFF3 files
gencode_Grch38_path = Path("resources/gencode.v45.basic.annotation.gff3.gz")
gencode_Grch37_path = Path("resources/gencode.v45lift37.basic.annotation.gff3.gz")

# Read the GRCh38 GFF3 file into a DataFrame
grch38_df = read_gff3_to_df(gencode_Grch38_path)

# Read the GRCh37 GFF3 file into a DataFrame
grch37_df = read_gff3_to_df(gencode_Grch37_path)


In [44]:
# This is the cell used to create the CDS DataFrame and save it to a parquet file.
cds_grch38_df=grch38_df[grch38_df["Feature Type"]=="CDS"].copy()
cds_grch38_df["Genome_Build"]="GRCh38" 


cds_grch37_df=grch37_df[grch37_df["Feature Type"]=="CDS"].copy()
cds_grch37_df["Genome_Build"]="GRCh37"

cds_dfs=pd.concat([cds_grch38_df,cds_grch37_df])

cds_dfs.reset_index(drop=True, inplace=True)

cds_dfs.to_parquet("resources/genecode_CDS_DataFrame.parquet")


In [37]:
cds_dfs

,Sequence ID,source,Feature Type,Feature Start,Feature End,Score,Strand,Phase,ID,gene_id,...,artif_dupl,Genome_Build,remap_status,remap_num_mappings,remap_target_status,remap_original_location,gene_status,remap_substituted_missing_target,transcript_status,remap_original_id
0,chr1,HAVANA,CDS,65565,65573,.,+,0,CDS:ENST00000641515.2,ENSG00000186092.7,...,NaN,GRCh38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,CDS,69037,70008,.,+,0,CDS:ENST00000641515.2,ENSG00000186092.7,...,NaN,GRCh38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,CDS,450740,451678,.,-,0,CDS:ENST00000426406.4,ENSG00000284733.2,...,NaN,GRCh38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,HAVANA,CDS,685716,686654,.,-,0,CDS:ENST00000332831.5,ENSG00000284662.2,...,NaN,GRCh38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,HAVANA,CDS,924432,924948,.,+,0,CDS:ENST00000616016.5,ENSG00000187634.13,...,NaN,GRCh38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311174,chrM,ENSEMBL,CDS,10470,10766,.,+,0,CDS:ENST00000361335.1,ENSG00000212907.2_2,...,NaN,GRCh37,full_contig,NaN,NaN,chrM:+:10470-10766,NaN,NaN,NaN,NaN
1311175,chrM,ENSEMBL,CDS,10760,12137,.,+,0,CDS:ENST00000361381.2,ENSG00000198886.2_2,...,NaN,GRCh37,full_contig,NaN,NaN,chrM:+:10760-12137,NaN,NaN,NaN,NaN
1311176,chrM,ENSEMBL,CDS,12337,14148,.,+,0,CDS:ENST00000361567.2,ENSG00000198786.2_2,...,NaN,GRCh37,full_contig,NaN,NaN,chrM:+:12337-14148,NaN,NaN,NaN,NaN
1311177,chrM,ENSEMBL,CDS,14149,14673,.,-,0,CDS:ENST00000361681.2,ENSG00000198695.2_4,...,NaN,GRCh37,full_contig,NaN,NaN,chrM:-:14149-14673,NaN,NaN,NaN,NaN


In [57]:
def convert_ProteinLoc_to_GenomicLoc(CDS_df: pd.DataFrame, transcript_id: str, protein_start: int, protein_end: int, genome_build: str) -> list:
    if genome_build == "hg19":
        genome_build = "GRCh37"
    elif genome_build == "hg38":
        genome_build = "GRCh38"
        
    transcript_df=CDS_df[CDS_df["transcript_id"].str.startswith(transcript_id) & (CDS_df["Genome_Build"]== genome_build)].copy()

    if transcript_df.empty:
        logging.error("No transcript found")
        return []
    elif transcript_df.iloc[0]["Strand"]=="+":
        transcript_df.sort_values("Feature Start",inplace=True)
    elif transcript_df.iloc[0]["Strand"]=="-":
        transcript_df.sort_values("Feature Start",inplace=True,ascending=False)

    transcript_df["length"]=transcript_df["Feature End"].astype(int)-transcript_df["Feature Start"].astype(int)+1
    transcript_df['CDS_start'] = (transcript_df.groupby('protein_id')['length'].cumsum() - transcript_df['length'])
    transcript_df['CDS_end'] = transcript_df.groupby('protein_id')['length'].cumsum()
    
    CDS_start, CDS_end = protein_start*3, protein_end*3

    overlap_CDS = transcript_df[(transcript_df["CDS_end"] >= CDS_start) & (transcript_df["CDS_start"] <= CDS_end)]

    if overlap_CDS.empty:
        logging.error("No overlap found")
        return[]
    elif overlap_CDS.iloc[0]["Strand"] == "-":
        chromosome = overlap_CDS.iloc[0]["Sequence ID"]
        genomic_start = int(overlap_CDS.iloc[-1]["Feature End"]) - (CDS_end - overlap_CDS.iloc[-1]["CDS_start"])+1
        genomic_end = int(overlap_CDS.iloc[0]["Feature End"]) - (CDS_start - overlap_CDS.iloc[0]["CDS_start"])+3
    elif overlap_CDS.iloc[0]["Strand"] == "+":
        chromosome = overlap_CDS.iloc[0]["Sequence ID"]
        genomic_start = int(overlap_CDS.iloc[0]["Feature Start"]) + (CDS_start - overlap_CDS.iloc[0]["CDS_start"])-3
        genomic_end = int(overlap_CDS.iloc[-1]["Feature Start"]) + (CDS_end - overlap_CDS.iloc[-1]["CDS_start"])-1
    else:
        logging.error("Format Error No strand info found in fearture file")
        return []
    
    return (chromosome,genomic_start,genomic_end)

In [58]:
test_input = StringIO("""Gene Symbol	Transcript ID	Genome Build	Protein Domains
CIC	ENST00000575354	GRCh37	High mobility groupbox domain: 199—269
CIC	ENST00000681038	GRCh38	High mobility groupbox domain : 199-269
BRCA1	ENST00000471181	GRCh37	Zincfinger, RING—type: 24—658;BRCA1 , serine—richdomain: 345—507
BRCA1	ENST00000357654	GRCh38	Zincfinger, RING—type: 24—658;BRCA1 , serine—richdomain: 345—507""")
protein_data = pd.read_csv(test_input, sep='\t')

output_list=list()
for index, row in protein_data.iterrows():
    for item in row["Protein Domains"].split(";"):
        
        domain_dict=dict()
        domain_dict["Domain Name"],domain_dict["protein_pos"]=item.split(":")
        domain_dict["protein_start"],domain_dict["protein_end"]=list(domain_dict["protein_pos"].replace("—","-").strip().split("-"))
        domain_dict.pop("protein_pos")

        genomic_loc=convert_ProteinLoc_to_GenomicLoc(cds_dfs,transcript_id=row["Transcript ID"],protein_start=int(domain_dict["protein_start"]),protein_end=int(domain_dict["protein_end"]),genome_build=row["Genome Build"])
        
        domain_dict["Gene Symbol"]=row["Gene Symbol"]
        domain_dict["Genome_Build"]=row["Genome Build"]
        domain_dict["Chrom"]=genomic_loc[0]
        domain_dict["Domain Coordinates"]=domain_dict["protein_start"]+"-"+domain_dict["protein_end"]
        domain_dict["AA Length"]=int(domain_dict["protein_end"])-int(domain_dict["protein_start"])+1
        domain_dict["Genomic Coordiantes"]=f"{genomic_loc[1]}-{genomic_loc[2]}"
        domain_dict["NUC Length"]=int(genomic_loc[2])-int(genomic_loc[1])+1
        
        output_list.append(domain_dict)
    

    
    

In [61]:
pd.DataFrame(output_list)[["Gene Symbol", "Genome_Build", "Chrom", "Domain Name", "Domain Coordinates", "AA Length", "Genomic Coordiantes", "NUC Length"]]


,Gene Symbol,Genome_Build,Chrom,Domain Name,Domain Coordinates,AA Length,Genomic Coordiantes,NUC Length
0,CIC,GRCh37,chr19,High mobility groupbox domain,199-269,71,42791709-42792003,295
1,CIC,GRCh38,chr19,High mobility groupbox domain,199-269,71,42272378-42272590,213
2,BRCA1,GRCh37,chr17,"Zincfinger, RING—type",24-658,635,41245574-41276044,30471
3,BRCA1,GRCh37,chr17,"BRCA1 , serine—richdomain",345-507,163,41246027-41246515,489
4,BRCA1,GRCh38,chr17,"Zincfinger, RING—type",24-658,635,43093557-43124027,30471
5,BRCA1,GRCh38,chr17,"BRCA1 , serine—richdomain",345-507,163,43094010-43094498,489


In [54]:
transcript_id="ENST00000575354"
genome_build="GRCh37"
cds_dfs[cds_dfs["transcript_id"].str.startswith(transcript_id) & (cds_dfs["Genome_Build"]== genome_build)]

,Sequence ID,source,Feature Type,Feature Start,Feature End,Score,Strand,Phase,ID,gene_id,...,artif_dupl,Genome_Build,remap_status,remap_num_mappings,remap_target_status,remap_original_location,gene_status,remap_substituted_missing_target,transcript_status,remap_original_id
1236886,chr19,HAVANA,CDS,42788857,42788923,.,+,0,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42284705-42284771,NaN,NaN,NaN,NaN
1236887,chr19,HAVANA,CDS,42790923,42791072,.,+,2,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42286771-42286920,NaN,NaN,NaN,NaN
1236888,chr19,HAVANA,CDS,42791158,42791392,.,+,2,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42287006-42287240,NaN,NaN,NaN,NaN
1236889,chr19,HAVANA,CDS,42791472,42791601,.,+,1,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42287320-42287449,NaN,NaN,NaN,NaN
1236890,chr19,HAVANA,CDS,42791697,42791879,.,+,0,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42287545-42287727,NaN,NaN,NaN,NaN
1236891,chr19,HAVANA,CDS,42791962,42792127,.,+,0,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42287810-42287975,NaN,NaN,NaN,NaN
1236892,chr19,HAVANA,CDS,42793040,42793242,.,+,2,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42288888-42289090,NaN,NaN,NaN,NaN
1236893,chr19,HAVANA,CDS,42793333,42793558,.,+,0,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42289181-42289406,NaN,NaN,NaN,NaN
1236894,chr19,HAVANA,CDS,42794000,42794103,.,+,2,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42289848-42289951,NaN,NaN,NaN,NaN
1236895,chr19,HAVANA,CDS,42794385,42795618,.,+,0,CDS:ENST00000575354.6,ENSG00000079432.9_16,...,NaN,GRCh37,full_contig,NaN,NaN,chr19:+:42290233-42291466,NaN,NaN,NaN,NaN
